<div class="alert alert-block alert-info">
<b>Deadline:</b> May 4, 2022 (Wednesday) 23:00
</div>

# Exercise 2. Generative adversarial networks (GANs). WGAN-GP: Wasserstein GAN with gradient penalty

The goal of this exercise is to get familiar with WGAN-GP: one of the most popular versions of GANs, which is relatively easy to train.

The algorithm was introduced in the paper [Improved Training of Wasserstein GANs](https://arxiv.org/pdf/1704.00028.pdf).

In [45]:
skip_training = True  # Set this flag to True before validation and submission

In [46]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [47]:
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

import torch
import torchvision
import torch.nn as nn
from torch.nn import functional as F
import torchvision.transforms as transforms
import torchvision.utils as utils

import tools
import tests

In [48]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [49]:
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [50]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

# Data

We will use MNIST data in this exercise. Note that we re-scale images so that the pixel intensities are in the range [-1, 1].

In [51]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Normalize((0.5,), (0.5,))  # Scale to [-1, 1]
])

trainset = torchvision.datasets.MNIST(root=data_dir, train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

# Wasserstein GAN (WGAN)

The WGAN value function is constructed as
$$
  \min_g \max_{d \in D} E_{x∼P_r}[d(x)] − E_{\widetilde x∼P_g}[d(\widetilde{x})]
$$
where
* the discriminator $d$ is constrained to be in a restricted set $D$ of functions
* $P_r$ is the data distribution
* $P_g$ is the model distribution. Samples from the model distribution are produced as follows:
\begin{align}
z &\sim N(0, I)
\\
\widetilde{x} &= g(z)
\end{align}

## Generator

Implement the generator in the cell below. We recommend you to use the same architecture of the generator as in Exercise 11.1.

In [52]:
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        """WGAN generator.
        
        Args:
          nz:  Number of elements in the latent code.
          ngf: Base size (number of channels) of the generator layers.
          nc:  Number of channels in the generated images.
        """
        super(Generator, self).__init__()
        
        # YOUR CODE HERE
        self.gen_conv1 = nn.ConvTranspose2d(kernel_size = 4, stride = 2,in_channels = nz, out_channels = 4*ngf, bias = False, padding = 0)
        self.norm1 = nn.BatchNorm2d(num_features = 4*ngf )
        self.relu1 = nn.ReLU()
        
        # ConvTranspose2d layer with kernel_size=4, stride=2, 2*ngf output channels, no bias, followed by BatchNorm2d and ReLU
        self.gen_conv2 = nn.ConvTranspose2d(kernel_size = 4, stride = 2,in_channels = 4*ngf, out_channels = 2*ngf, bias = False, padding = 1)
        self.norm2 = nn.BatchNorm2d(num_features = 2*ngf )
        self.relu2 = nn.ReLU()
        
        # ConvTranspose2d layer with kernel_size=4, stride=2, ngf output channels, no bias, followed by BatchNorm2d and ReLU
        self.gen_conv3 = nn.ConvTranspose2d(kernel_size = 4, stride = 2, in_channels = 2*ngf, out_channels = ngf, bias = False, padding = 2)
        self.norm3 = nn.BatchNorm2d(num_features = ngf )
        self.relu3 = nn.ReLU()
        
        # ConvTranspose2d layer with kernel_size=4, stride=2, nc output channels, no bias, followed by tanh
        self.gen_conv4 = nn.ConvTranspose2d(kernel_size = 4, stride = 2, in_channels = ngf, out_channels = nc, bias = False, padding = 1)
        self.tanh = nn.Tanh()

    def forward(self, z, verbose=False):
        """Generate images by transforming the given noise tensor.
        
        Args:
          z of shape (batch_size, nz, 1, 1): Tensor of noise samples. We use the last two singleton dimensions
              so that we can feed z to the generator without reshaping.
          verbose (bool): Whether to print intermediate shapes (True) or not (False).
        
        Returns:
          out of shape (batch_size, nc, 28, 28): Generated images.
        """
        # YOUR CODE HERE
        if verbose: print(z.size())
            
        out = self.relu1(self.norm1(self.gen_conv1(z)))
        if verbose: print(out.size())
            
        out = self.relu2(self.norm2(self.gen_conv2(out)))
        if verbose: print(out.size())
            
        out = self.relu3(self.norm3(self.gen_conv3(out)))
        if verbose: print(out.size())
        
        out = self.tanh(self.gen_conv4(out))
        if verbose: print(out.size())
            
        return out

In [53]:
def test_Generator_shapes():
    batch_size = 32
    nz = 10
    netG = Generator(nz, ngf=64, nc=1)

    noise = torch.randn(batch_size, nz, 1, 1)
    out = netG(noise, verbose=True)

    assert out.shape == torch.Size([batch_size, 1, 28, 28]), f"Bad out.shape: {out.shape}"
    print('Success')

test_Generator_shapes()

torch.Size([32, 10, 1, 1])
torch.Size([32, 256, 4, 4])
torch.Size([32, 128, 8, 8])
torch.Size([32, 64, 14, 14])
torch.Size([32, 1, 28, 28])
Success


### Loss for training the generator

The generator is trained to minimize the relevant part of the value function using a fixed discriminator $d$:
$$
  \min_g − E_{\widetilde{x} \sim P_g}[d(\widetilde x)]
$$

In [54]:
def generator_loss(netD, fake_images):
    """Loss computed to train the WGAN generator.

    Args:
      netD: The discriminator whose forward function takes inputs of shape (batch_size, nc, 28, 28)
         and produces outputs of shape (batch_size, 1).
      fake_images of shape (batch_size, nc, 28, 28): Fake images produces by the generator.

    Returns:
      loss: The relevant part of the WGAN value function.
    """
    # YOUR CODE HERE
    fake_images = fake_images.to(device)
    
    out = netD.forward(fake_images).reshape(-1)
    
    loss = -torch.mean(out)
    
    return loss

In [55]:
# This cell tests generator_loss()

## Discriminator

Implement the WGAN discriminator in the cell below. You can use almost the same architecture as the architecture of the discriminator in Exercise 11.1. The difference is that there is no need to use `sigmoid` nonlinearity in the output layer because the output of the discriminator does not have to be between 0 and 1.

In [56]:
class Discriminator(nn.Module):
    def __init__(self, nc=1, ndf=64):
        """
        Args:
          nc:  Number of channels in the images.
          ndf: Base size (number of channels) of the discriminator layers.
        """
        # YOUR CODE HERE
        super(Discriminator, self).__init__()
        
        # Conv2d layer with kernel_size=4, stride=2, ndf output channels, no bias, followed by BatchNorm2d and LeakyReLU(0.2)
        self.dis_conv1 = nn.Conv2d(in_channels=nc, out_channels=ndf, kernel_size=4, stride=2, bias=False, padding=1)
        self.norm1 = nn.BatchNorm2d(num_features = ndf )
        self.relu1 = nn.LeakyReLU(0.2)
        
        # Conv2d layer with kernel_size=4, stride=2, ndf output channels, no bias, followed by BatchNorm2d and LeakyReLU(0.2)
        self.dis_conv2 = nn.Conv2d(in_channels=ndf, out_channels=2*ndf, kernel_size=4, stride=2, bias=False, padding=2)
        self.norm2 = nn.BatchNorm2d(num_features = 2*ndf )
        self.relu2 = nn.LeakyReLU(0.2)
        
        # Conv2d layer with kernel_size=4, stride=2, 4*ndf output channels, no bias, followed by BatchNorm2d and LeakyReLU(0.2)
        self.dis_conv3 = nn.Conv2d(in_channels=2*ndf, out_channels=4*ndf, kernel_size=4, stride=2, bias=False, padding=1)
        self.norm3 = nn.BatchNorm2d(num_features = 4*ndf )
        self.relu3 = nn.LeakyReLU(0.2)
        
        # Conv2d layer with kernel_size=4, stride=2, 1 output channel, no bias, followed by sigmoid.
        self.dis_conv4 = nn.Conv2d(in_channels=4*ndf, out_channels=1, kernel_size=4, stride=2, bias=False, padding=0)

        
    def forward(self, x, verbose=False):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Images to be evaluated.
        
        Returns:
          out of shape (batch_size,): Discriminator outputs for images x.
        """
        # YOUR CODE HERE
        if verbose: print(x.size())
        
        out = self.relu1(self.norm1(self.dis_conv1(x)))
        if verbose: print(out.size())
        
        out = self.relu2(self.norm2(self.dis_conv2(out)))
        if verbose: print(out.size())
        
        out = self.relu3(self.norm3(self.dis_conv3(out)))
        if verbose: print(out.size())
            
        out = self.dis_conv4(out)
        if verbose: print(out.size())
            
        out = torch.squeeze(out)
        if verbose: print(out.size())
        
        return out

In [57]:
def test_Discriminator_shapes():
    nz = 10  # size of the latent z vector
    netD = Discriminator(nc=1, ndf=64)

    batch_size = 32
    images = torch.ones(batch_size, 1, 28, 28)
    out = netD(images, verbose=True)
    assert out.shape == torch.Size([batch_size]), f"Bad out.shape: {out.shape}"
    print('Success')

test_Discriminator_shapes()

torch.Size([32, 1, 28, 28])
torch.Size([32, 64, 14, 14])
torch.Size([32, 128, 8, 8])
torch.Size([32, 256, 4, 4])
torch.Size([32, 1, 1, 1])
torch.Size([32])
Success


### Loss for training the WGAN discriminator

Recall the value function of WGAN:
$$
  \min_g \max_{d \in D} E_{x∼P_r}[d(x)] − E_{\widetilde x∼P_g}[d(\widetilde x)]
$$
To tune the discriminator, we need to minimize the following function:
$$
  \min_{d \in D} - E_{x∼P_r}[d(x)] + E_{\widetilde x∼P_g}[d(\widetilde x)]
$$
You need to implement this loss function *assuming no constraints on d* in the function below.

In [58]:
def discriminator_loss(netD, real_images, fake_images):
    """
    Args:
      netD: The discriminator.
      real_images of shape (batch_size, nc, 28, 28): Real images.
      fake_images of shape (batch_size, nc, 28, 28): Fake images.

    Returns:
      loss (scalar tensor): Loss for training the WGAN discriminator.
    """
    # YOUR CODE HERE
    real_images = real_images.to(device)
    fake_images = fake_images.to(device)

    real = netD.forward(real_images).reshape(-1)
    fake = netD.forward(fake_images).reshape(-1)
    
    loss = - torch.mean(real) + torch.mean(fake)
    
    return loss

In [59]:
# This cell tests discriminator_loss()

Without constraints on $d$, the WGAN value function can be made infinitely large. WGAN constrains the derivative of $d$ using a gradient penalty. The penalty is computed at random points between real images and generated ones using the following procedure:
* Given a real image $x$ and a fake image $\widetilde x$, draw a random number $\epsilon \sim U[0,1]$
* $\hat{x} \leftarrow \epsilon x + (1−\epsilon) \widetilde x$
* Compute the gradient penalty $(‖\nabla_{\hat{x}} d(\hat{x})‖_2−1)^2$
where $\nabla_{\hat{x}} d(\hat{x})$ is the gradient of $d$ computed at $\hat{x}$.

Your task is to implement the gradient penalty in the cell below.

Notes:

* We need to compute the gradient $\nabla d$ so that we can differentiate through the gradient when computing the derivatives wrt the parameters of the discriminator. This can be achieved by using function [torch.autograd.grad](https://pytorch.org/docs/stable/autograd.html#torch.autograd.grad) which can create a computational graph with the gradient computations.
* The gradient penalty is the average of $(‖\nabla_{\hat{x}} d(\hat{x})‖_2−1)^2$ computed across all samples.
* The second output returned by the function is needed for testing your implementation.

In [60]:
def gradient_penalty(netD, real, fake_detached):
    """
    Args:
      netD: The discriminator.
      real of shape (batch_size, nc, 28, 28): Real images.
      fake_detached of shape (batch_size, nc, 28, 28): Fake images (detached from the computational graph).

    Returns:
      grad_penalty (scalar tensor): Gradient penalty.
      x of shape (batch_size, nc, 28, 28): Points x-hat in which the gradient penalty is computed.
    """
    # YOUR CODE HERE
    batch_size = real.shape[0]
    nc = real.shape[1]
    
    eps = torch.rand((batch_size, 1, 1, 1)).repeat(1, nc, 28, 28).to(fake_detached.device)
    
    x = eps * real + ((1 - eps) * fake_detached)   
    x = torch.autograd.Variable(x, requires_grad=True)
    
    out = netD(x)
    
    gradients = torch.autograd.grad(inputs=x, outputs=out,
                                    grad_outputs=torch.ones_like(out), 
                                    create_graph=True, retain_graph=True)[0]
    
    gradients = gradients.view(gradients.shape[0], -1)
    gradients_norm = gradients.norm(2, dim=1)
    
    gradient_penalty = torch.mean((gradients_norm - 1).pow(2))
    
    return gradient_penalty, x

In [61]:
# This cell test gradient_penalty()

# Training WGAN-GP

We will now train WGAN-GP. To assess the quality of the generated samples, we will again use the FD score.

In [62]:
import fd

# Load an FD scorer pre-trained on MNIST
fdscore = fd.FDScore.pretrained()
fdscore.to(device)

FDScore(
  (net): Classifier(
    (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (fc1): Linear(in_features=512, out_features=120, bias=True)
    (fc2): Linear(in_features=120, out_features=84, bias=True)
    (fc3): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [63]:
# Create the network
nz = 10
netG = Generator(nz=nz, ngf=128, nc=1).to(device)
netD = Discriminator(nc=1, ndf=128).to(device)

### Training loop

Implement the training loop in the cell below. The recommended hyperparameters:
* Optimizer of the discriminator: Adam with learning rate 0.0002 and `betas=(0.5, 0.999)`
* Optimizer of the generator:     Adam with learning rate 0.0002 and `betas=(0.5, 0.999)`
* Weight $\lambda=10$ of the gradient penalty term in the discriminator loss:
$$
  \min_{d} - E_{x∼P_r}[D(x)] + E_{\tilde x∼P_g}[d(\widetilde x)]
  + \lambda (‖\nabla_{\hat{x}} d(\hat{x})‖_2−1)^2
$$

Hints:
- We will use the FD score to assess the quality of the generated samples. The desired level of 10 should be reached after 5-10 epochs. Note that the score is stochastic and it can fluctuate during training. At convergence, the FD score can fluctuate in the range [4, 10].
- You can use the following code to track the training progress. The code plots some generated images and computes the score that we use to evaluate the trained model. Note that the images fed to the scorer need to be normalized to be in the range [-1, 1].
```python
with torch.no_grad():
    # Plot generated images
    z = torch.randn(144, nz, 1, 1, device=device)
    samples = netG(z)
    tools.plot_generated_samples(samples)
    
    # Compute score
    z = torch.randn(1000, nz, 1, 1, device=device)
    samples = netG(z)
    score = fdscore.calculate(samples)
```
- The quality of the images is slightly worse than with the DCGAN.

In [64]:
if not skip_training:
    # YOUR CODE HERE
    print('Start training')
    # YOUR CODE HERE
    lamb = 10
    optim_G = torch.optim.Adam(netG.parameters(), lr=0.0001, betas=(0.5,0.999))
    optim_D = torch.optim.Adam(netD.parameters(), lr=0.0001, betas=(0.5,0.999))
    for epoch in range(20):
        for i, data in enumerate(dataloader, 0):

            images, labels = data
            images = images.to(device)
            labels.to(device)
            iamges = images * 2 - 1 

            z = torch.randn(images.size(0), nz, 1, 1, device=device)
            gen_images = netG(z)
            
            optim_D.zero_grad()


            penalty, x_hat = gradient_penalty(netD, images, gen_images.detach())
            loss = discriminator_loss(netD, images, gen_images.detach()) + lamb * penalty
            loss.backward()

            optim_D.step()
            
            optim_G.zero_grad()
            
            g_loss = generator_loss(netD, gen_images)
            g_loss.backward()

            optim_G.step()

        print('\nValidationm in Epoch {}:'.format(epoch))
        with torch.no_grad():
            # Plot generated images
            #z = torch.randn(144, nz, 1, 1, device=device)
            #samples = netG(z)
            #tools.plot_generated_samples(samples)
            
            # Compute score
            z = torch.randn(1000, nz, 1, 1, device=device)
            samples = netG(z)
            samples = (samples + 1) / 2  # Re-normalize to [0, 1]
            score = scorer(samples)
        print(score.item())
        if epoch > 15 and score > 0.7:
            break

In [65]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(netG, '2_wgan_g.pth', confirm=True)
    tools.save_model(netD, '2_wgan_d.pth', confirm=True)

In [66]:
if skip_training:
    nz = 10
    netG = Generator(nz=nz, ngf=128, nc=1)
    netD = Discriminator(nc=1, ndf=128)
    
    tools.load_model(netG, '2_wgan_g.pth', device)
    tools.load_model(netD, '2_wgan_d.pth', device)

RuntimeError: Error(s) in loading state_dict for Generator:
	Missing key(s) in state_dict: "gen_conv1.weight", "norm1.weight", "norm1.bias", "norm1.running_mean", "norm1.running_var", "gen_conv2.weight", "norm2.weight", "norm2.bias", "norm2.running_mean", "norm2.running_var", "gen_conv3.weight", "norm3.weight", "norm3.bias", "norm3.running_mean", "norm3.running_var", "gen_conv4.weight". 
	Unexpected key(s) in state_dict: "conv1.weight", "batch1.weight", "batch1.bias", "batch1.running_mean", "batch1.running_var", "batch1.num_batches_tracked", "conv2.weight", "batch2.weight", "batch2.bias", "batch2.running_mean", "batch2.running_var", "batch2.num_batches_tracked", "conv3.weight", "batch3.weight", "batch3.bias", "batch3.running_mean", "batch3.running_var", "batch3.num_batches_tracked", "conv4.weight". 

## GAN evaluation

In [ ]:
# Save generated samples (the pth-files will be submitted automatically together with your notebook)
if not skip_training:
    with torch.no_grad():
        z = torch.randn(144, nz, 1, 1, device=device)
        samples = netG(z)
        torch.save(samples, '2_wgan_samples.pth')
else:
    samples = torch.load('2_wgan_samples.pth', map_location=lambda storage, loc: storage)

tools.plot_generated_samples(samples)

In [ ]:
# Compute the FD score
torch.manual_seed(0)
with torch.no_grad():
    z = torch.randn(1000, nz, 1, 1, device=device)
    samples = netG(z)
    tools.plot_generated_samples(samples[:144])
    score = fdscore.calculate(samples)

print(f'FD score: {score:.5f}')

In [ ]:
# This cell is used for grading

<div class="alert alert-block alert-info">
<b>Conclusion</b>
</div>

In this notebook, we learned how to train Wasserstein GAN with gradient penalty.